## Iteration 2

In the second iteration, it was decided to add one more task type - quiz (the task field in the dataset was renamed to the description, and an additional type field was added), and add some additional rules that the model should follow. 

Right now, the model receives a request to generate a task or a quiz for a specific topic.

If it is a task:  
- Description: a problem description with a single question asked.
- Solution: an ordered list with a step-by-step explanation of how to get the correct answer.  
- Answer: Just a number, or a final equation.  

If it is a quiz:  
- Description: a problem and a set of options to choose from (always four options enumerated as Latin letters A, B, C, D).   
- Solution: an ordered list with a step-by-step explanation of how to get the correct answer.  
- Answer: Just a Latin letter corresponding to the answer.  

Also, the response needs to be formatted as a valid JSON right now.

The model had a few issues, all related to JSON formatting (adding an ordered list not as a string but as a JSON list, wrapping JSON in a markdown block, and so on). All solved with system prompt modification. 

In [43]:
import json
import jsonlines
import pandas as pd
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

In [44]:
pd.set_option('display.max_colwidth', None)

In [45]:
load_dotenv()

True

In [46]:
with jsonlines.open('dataset.jsonl') as reader:
    dataset = list(reader)

dataset[0]

{'topic': 'Арифметичні дії',
 'type': 'task',
 'description': 'Обчисліть значення виразу: $100 - (15 + 5) \\times 4$.',
 'solution': '1. Виконайте дію в дужках: $15 + 5 = 20$. 2. Виконайте множення отриманого результату на 4: $20 \\times 4 = 80$. 3. Виконайте віднімання від 100: $100 - 80 = 20$.',
 'answer': '20'}

In [47]:
system_prompt = """Ти - вчитель математики. Створи математичний контент на задану тему відповідно до вказаного типу.

Типи контенту:
1. Задача: 
   - description: умова задачі з одним питанням
   - solution: **нумерований** список кроків розв'язання
   - answer: число або рівняння

2. Тест:
   - description: умова задачі з чотирма варіантами відповіді (A, B, C, D)
   - solution: **нумерований** список кроків для знаходження відповіді
   - answer: одна латинська літера (A, B, C або D)

Відповідь надай у JSON форматі з трьома аттрибутами типу string: description, solution, answer. Не обгортай JSON у додаткові спецсимволи."""

In [48]:
model = ChatOpenAI(model="gpt-4o", temperature=0.1)

In [49]:
type_map = {'task': 'Задача', 'quiz': 'Тест'}

for item in dataset[:10]:

    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Тема: {item['topic']}, Тип: {type_map[item['type']]}")
    ]

    response = model.invoke(messages)
    
    try:
        parsed = json.loads(response.content)
        is_valid_json = True
        generated_description = parsed.get('description', '')
        generated_solution = parsed.get('solution', '')
        generated_answer = parsed.get('answer', '')
    except json.JSONDecodeError:
        is_valid_json = False
        generated_description = ''
        generated_solution = ''
        generated_answer = ''
    
    display(pd.DataFrame([
        {'field': 'topic', 'value': item['topic']},
        {'field': 'type', 'value': item['type']},
        {'field': 'system_prompt', 'value': messages[0].content},
        {'field': 'user_message', 'value': messages[1].content},
        {'field': 'model_response', 'value': response.content},
        {'field': 'sample_description', 'value': item['description']},
        {'field': 'sample_solution', 'value': item['solution']},
        {'field': 'sample_answer', 'value': item['answer']},
        {'field': 'is_valid_json', 'value': is_valid_json},
        {'field': 'generated_description', 'value': generated_description},
        {'field': 'generated_solution', 'value': generated_solution},
        {'field': 'generated_answer', 'value': generated_answer},
    ]))

,field,value
0,topic,Арифметичні дії
1,type,task
2,system_prompt,"Ти - вчитель математики. Створи математичний контент на задану тему відповідно до вказаного типу.\n\nТипи контенту:\n1. Задача: \n - description: умова задачі з одним питанням\n - solution: **нумерований** список кроків розв'язання\n - answer: число або рівняння\n\n2. Тест:\n - description: умова задачі з чотирма варіантами відповіді (A, B, C, D)\n - solution: **нумерований** список кроків для знаходження відповіді\n - answer: одна латинська літера (A, B, C або D)\n\nВідповідь надай у JSON форматі з трьома аттрибутами типу string: description, solution, answer. Не обгортай JSON у додаткові спецсимволи."
3,user_message,"Тема: Арифметичні дії, Тип: Задача"
4,model_response,"{\n ""description"": ""У магазині продають яблука за ціною 15 гривень за кілограм. Олена купила 3 кілограми яблук. Скільки грошей вона витратила?"",\n ""solution"": ""1. Визначити ціну за один кілограм яблук, яка дорівнює 15 гривень.\n2. Визначити кількість кілограмів яблук, яку купила Олена, що дорівнює 3 кілограми.\n3. Помножити ціну за один кілограм на кількість кілограмів: 15 гривень/кг * 3 кг = 45 гривень."",\n ""answer"": ""45""\n}"
5,sample_description,Обчисліть значення виразу: $100 - (15 + 5) \times 4$.
6,sample_solution,1. Виконайте дію в дужках: $15 + 5 = 20$. 2. Виконайте множення отриманого результату на 4: $20 \times 4 = 80$. 3. Виконайте віднімання від 100: $100 - 80 = 20$.
7,sample_answer,20
8,is_valid_json,True
9,generated_description,У магазині продають яблука за ціною 15 гривень за кілограм. Олена купила 3 кілограми яблук. Скільки грошей вона витратила?


,field,value
0,topic,Геометрія
1,type,quiz
2,system_prompt,"Ти - вчитель математики. Створи математичний контент на задану тему відповідно до вказаного типу.\n\nТипи контенту:\n1. Задача: \n - description: умова задачі з одним питанням\n - solution: **нумерований** список кроків розв'язання\n - answer: число або рівняння\n\n2. Тест:\n - description: умова задачі з чотирма варіантами відповіді (A, B, C, D)\n - solution: **нумерований** список кроків для знаходження відповіді\n - answer: одна латинська літера (A, B, C або D)\n\nВідповідь надай у JSON форматі з трьома аттрибутами типу string: description, solution, answer. Не обгортай JSON у додаткові спецсимволи."
3,user_message,"Тема: Геометрія, Тип: Тест"
4,model_response,"{\n ""description"": ""У трикутнику ABC, кут A дорівнює 60 градусів, кут B дорівнює 70 градусів. Який градусний вимір кута C?"",\n ""solution"": ""1. Відомо, що сума кутів у трикутнику дорівнює 180 градусів. \n2. Знайдемо кут C, використовуючи формулу: кут C = 180 - кут A - кут B. \n3. Підставимо відомі значення: кут C = 180 - 60 - 70. \n4. Обчислимо: кут C = 50 градусів."",\n ""answer"": ""C""\n}"
5,sample_description,Чому дорівнює сума внутрішніх кутів довільного трикутника? \n A) $90^\circ$ \n B) $180^\circ$ \n C) $270^\circ$ \n D) $360^\circ$
6,sample_solution,"1. Згадайте фундаментальну теорему геометрії про суму кутів трикутника. 2. Теорема стверджує, що сума кутів завжди дорівнює $180^\circ$. 3. Перевірте варіанти: A ($90^\circ$) — замало, C і D — забагато для трикутника. 4. Оберіть варіант B."
7,sample_answer,B
8,is_valid_json,True
9,generated_description,"У трикутнику ABC, кут A дорівнює 60 градусів, кут B дорівнює 70 градусів. Який градусний вимір кута C?"


,field,value
0,topic,Спрощення виразів
1,type,task
2,system_prompt,"Ти - вчитель математики. Створи математичний контент на задану тему відповідно до вказаного типу.\n\nТипи контенту:\n1. Задача: \n - description: умова задачі з одним питанням\n - solution: **нумерований** список кроків розв'язання\n - answer: число або рівняння\n\n2. Тест:\n - description: умова задачі з чотирма варіантами відповіді (A, B, C, D)\n - solution: **нумерований** список кроків для знаходження відповіді\n - answer: одна латинська літера (A, B, C або D)\n\nВідповідь надай у JSON форматі з трьома аттрибутами типу string: description, solution, answer. Не обгортай JSON у додаткові спецсимволи."
3,user_message,"Тема: Спрощення виразів, Тип: Задача"
4,model_response,"{\n ""description"": ""Спростіть вираз: 3(2x + 4) - 5(x - 2)."",\n ""solution"": ""1. Розкрийте дужки: 3(2x + 4) = 6x + 12 і -5(x - 2) = -5x + 10.\n2. Запишіть вираз без дужок: 6x + 12 - 5x + 10.\n3. Зведіть подібні члени: 6x - 5x = x і 12 + 10 = 22.\n4. Отримайте спрощений вираз: x + 22."",\n ""answer"": ""x + 22""\n}"
5,sample_description,"Спростіть вираз, використовуючи формулу різниці квадратів: $(x - 3)(x + 3)$."
6,sample_solution,"1. Згадайте формулу різниці квадратів: $(a - b)(a + b) = a^2 - b^2$. 2. Підставте значення з виразу, де $a = x$ і $b = 3$. 3. Обчисліть квадрати: $x^2 - 3^2 = x^2 - 9$."
7,sample_answer,$x^2 - 9$
8,is_valid_json,True
9,generated_description,Спростіть вираз: 3(2x + 4) - 5(x - 2).


,field,value
0,topic,Нерівності
1,type,quiz
2,system_prompt,"Ти - вчитель математики. Створи математичний контент на задану тему відповідно до вказаного типу.\n\nТипи контенту:\n1. Задача: \n - description: умова задачі з одним питанням\n - solution: **нумерований** список кроків розв'язання\n - answer: число або рівняння\n\n2. Тест:\n - description: умова задачі з чотирма варіантами відповіді (A, B, C, D)\n - solution: **нумерований** список кроків для знаходження відповіді\n - answer: одна латинська літера (A, B, C або D)\n\nВідповідь надай у JSON форматі з трьома аттрибутами типу string: description, solution, answer. Не обгортай JSON у додаткові спецсимволи."
3,user_message,"Тема: Нерівності, Тип: Тест"
4,model_response,"{\n ""description"": ""Розв'яжіть нерівність: 3x - 5 > 7. Яке з наведених значень x задовольняє цю нерівність? A) x = 2 B) x = 3 C) x = 4 D) x = 5"",\n ""solution"": ""1. Почнемо з розв'язання нерівності: 3x - 5 > 7.\n2. Додамо 5 до обох частин нерівності: 3x > 12.\n3. Поділимо обидві частини нерівності на 3: x > 4.\n4. Перевіримо кожен варіант відповіді:\n - A) x = 2: 2 не більше 4.\n - B) x = 3: 3 не більше 4.\n - C) x = 4: 4 не більше 4.\n - D) x = 5: 5 більше 4.\n5. Отже, правильна відповідь - D) x = 5."",\n ""answer"": ""D""\n}"
5,sample_description,Який проміжок є розв'язком нерівності $x > 5$? \n A) $(-\infty; 5)$ \n B) $[5; +\infty)$ \n C) $(5; +\infty)$ \n D) $(-\infty; 5]$
6,sample_solution,"1. Проаналізуйте знак нерівності: $>$ (строго більше) означає, що число 5 не входить у множину розв'язків, тому дужка біля 5 має бути круглою. 2. Визначте напрямок: $x$ більше 5, отже проміжок йде вправо до $+\infty$. 3. Порівняйте з варіантами: B має квадратну дужку, A і D йдуть у неправильному напрямку. 4. Оберіть варіант C, який відповідає умовам."
7,sample_answer,C
8,is_valid_json,True
9,generated_description,Розв'яжіть нерівність: 3x - 5 > 7. Яке з наведених значень x задовольняє цю нерівність? A) x = 2 B) x = 3 C) x = 4 D) x = 5


,field,value
0,topic,Системи рівнянь
1,type,task
2,system_prompt,"Ти - вчитель математики. Створи математичний контент на задану тему відповідно до вказаного типу.\n\nТипи контенту:\n1. Задача: \n - description: умова задачі з одним питанням\n - solution: **нумерований** список кроків розв'язання\n - answer: число або рівняння\n\n2. Тест:\n - description: умова задачі з чотирма варіантами відповіді (A, B, C, D)\n - solution: **нумерований** список кроків для знаходження відповіді\n - answer: одна латинська літера (A, B, C або D)\n\nВідповідь надай у JSON форматі з трьома аттрибутами типу string: description, solution, answer. Не обгортай JSON у додаткові спецсимволи."
3,user_message,"Тема: Системи рівнянь, Тип: Задача"
4,model_response,"{\n ""description"": ""Розв'яжіть систему рівнянь: \n1) 2x + 3y = 12 \n2) x - y = 1"",\n ""solution"": ""1. Візьмемо друге рівняння x - y = 1 і виразимо x через y: x = y + 1.\n2. Підставимо вираз для x у перше рівняння: 2(y + 1) + 3y = 12.\n3. Розкриємо дужки: 2y + 2 + 3y = 12.\n4. Об'єднаємо подібні члени: 5y + 2 = 12.\n5. Віднімемо 2 з обох сторін рівняння: 5y = 10.\n6. Поділимо обидві сторони на 5: y = 2.\n7. Підставимо значення y у вираз для x: x = 2 + 1 = 3."",\n ""answer"": ""(x, y) = (3, 2)""\n}"
5,sample_description,Розв'яжіть систему рівнянь: $\begin{cases} x + y = 10 \\ x - y = 4 \end{cases}$
6,sample_solution,"1. Застосуйте метод додавання рівнянь: $(x + y) + (x - y) = 10 + 4$. 2. Спростіть отримане рівняння: $2x = 14$. 3. Знайдіть $x$, поділивши обидві частини на 2: $x = 7$. 4. Підставте значення $x$ у перше рівняння ($7 + y = 10$) та знайдіть $y$: $y = 3$."
7,sample_answer,"$x = 7, y = 3$"
8,is_valid_json,True
9,generated_description,Розв'яжіть систему рівнянь: \n1) 2x + 3y = 12 \n2) x - y = 1


,field,value
0,topic,Квадратні корені
1,type,quiz
2,system_prompt,"Ти - вчитель математики. Створи математичний контент на задану тему відповідно до вказаного типу.\n\nТипи контенту:\n1. Задача: \n - description: умова задачі з одним питанням\n - solution: **нумерований** список кроків розв'язання\n - answer: число або рівняння\n\n2. Тест:\n - description: умова задачі з чотирма варіантами відповіді (A, B, C, D)\n - solution: **нумерований** список кроків для знаходження відповіді\n - answer: одна латинська літера (A, B, C або D)\n\nВідповідь надай у JSON форматі з трьома аттрибутами типу string: description, solution, answer. Не обгортай JSON у додаткові спецсимволи."
3,user_message,"Тема: Квадратні корені, Тип: Тест"
4,model_response,"{\n ""description"": ""Знайдіть значення виразу: √(49) + √(16). Яке з наведених значень є правильним? A) 11 B) 13 C) 15 D) 17"",\n ""solution"": ""1. Обчисліть квадратний корінь з 49: √(49) = 7. \n2. Обчисліть квадратний корінь з 16: √(16) = 4. \n3. Додайте отримані значення: 7 + 4 = 11."",\n ""answer"": ""A""\n}"
5,sample_description,Обчисліть значення виразу $\sqrt{144}$. \n A) 10 \n B) 11 \n C) 12 \n D) 14
6,sample_solution,"1. Визначте, що потрібно знайти число, яке при множенні саме на себе дає 144. 2. Перевірте варіанти відповідей піднесенням до квадрату. 3. $10^2=100$, $11^2=121$, $12^2=144$. 4. Оскільки $12^2=144$, оберіть варіант C."
7,sample_answer,C
8,is_valid_json,True
9,generated_description,Знайдіть значення виразу: √(49) + √(16). Яке з наведених значень є правильним? A) 11 B) 13 C) 15 D) 17


,field,value
0,topic,Арифметична прогресія
1,type,task
2,system_prompt,"Ти - вчитель математики. Створи математичний контент на задану тему відповідно до вказаного типу.\n\nТипи контенту:\n1. Задача: \n - description: умова задачі з одним питанням\n - solution: **нумерований** список кроків розв'язання\n - answer: число або рівняння\n\n2. Тест:\n - description: умова задачі з чотирма варіантами відповіді (A, B, C, D)\n - solution: **нумерований** список кроків для знаходження відповіді\n - answer: одна латинська літера (A, B, C або D)\n\nВідповідь надай у JSON форматі з трьома аттрибутами типу string: description, solution, answer. Не обгортай JSON у додаткові спецсимволи."
3,user_message,"Тема: Арифметична прогресія, Тип: Задача"
4,model_response,"{\n ""description"": ""Знайдіть суму перших 10 членів арифметичної прогресії, якщо перший член дорівнює 5, а різниця прогресії дорівнює 3."",\n ""solution"": ""1. Визначимо формулу для n-го члена арифметичної прогресії: a_n = a_1 + (n-1) * d.\n2. Знайдемо 10-й член прогресії: a_10 = 5 + (10-1) * 3 = 5 + 27 = 32.\n3. Використаємо формулу для суми перших n членів арифметичної прогресії: S_n = n/2 * (a_1 + a_n).\n4. Підставимо відомі значення: S_10 = 10/2 * (5 + 32) = 5 * 37 = 185."",\n ""answer"": ""185""\n}"
5,sample_description,"Дано арифметичну прогресію, де $a_1 = 5$ і різниця $d = 3$. Знайдіть п'ятий член прогресії ($a_5$)."
6,sample_solution,"1. Запишіть формулу $n$-го члена арифметичної прогресії: $a_n = a_1 + d(n - 1)$. 2. Підставте відомі значення ($a_1=5$, $d=3$, $n=5$) у формулу. 3. Виконайте обчислення в дужках: $5 - 1 = 4$. 4. Завершіть обчислення: $5 + 3 \times 4 = 5 + 12 = 17$."
7,sample_answer,17
8,is_valid_json,True
9,generated_description,"Знайдіть суму перших 10 членів арифметичної прогресії, якщо перший член дорівнює 5, а різниця прогресії дорівнює 3."


,field,value
0,topic,Тригонометрія
1,type,quiz
2,system_prompt,"Ти - вчитель математики. Створи математичний контент на задану тему відповідно до вказаного типу.\n\nТипи контенту:\n1. Задача: \n - description: умова задачі з одним питанням\n - solution: **нумерований** список кроків розв'язання\n - answer: число або рівняння\n\n2. Тест:\n - description: умова задачі з чотирма варіантами відповіді (A, B, C, D)\n - solution: **нумерований** список кроків для знаходження відповіді\n - answer: одна латинська літера (A, B, C або D)\n\nВідповідь надай у JSON форматі з трьома аттрибутами типу string: description, solution, answer. Не обгортай JSON у додаткові спецсимволи."
3,user_message,"Тема: Тригонометрія, Тип: Тест"
4,model_response,"{\n ""description"": ""Знайдіть значення виразу sin(30°) * cos(60°) + tan(45°). Варіанти відповідей: A) 1, B) 0.5, C) 1.5, D) 2"",\n ""solution"": ""1. Знайдіть значення sin(30°), яке дорівнює 0.5. \n2. Знайдіть значення cos(60°), яке також дорівнює 0.5. \n3. Обчисліть добуток sin(30°) * cos(60°) = 0.5 * 0.5 = 0.25. \n4. Знайдіть значення tan(45°), яке дорівнює 1. \n5. Додайте результати: 0.25 + 1 = 1.25."",\n ""answer"": ""C""\n}"
5,sample_description,Чому дорівнює значення $\sin 30^\circ$? \n A) $1$ \n B) $0.5$ \n C) $\frac{\sqrt{2}}{2}$ \n D) $\frac{\sqrt{3}}{2}$
6,sample_solution,"1. Зверніться до таблиці значень тригонометричних функцій. 2. Знайдіть значення для синуса кута $30^\circ$. 3. Табличне значення $\sin 30^\circ = 1/2$ або $0.5$. 4. Порівняйте з варіантами: C — це $45^\circ$, D — це $60^\circ$. 5. Оберіть варіант B."
7,sample_answer,B
8,is_valid_json,True
9,generated_description,"Знайдіть значення виразу sin(30°) * cos(60°) + tan(45°). Варіанти відповідей: A) 1, B) 0.5, C) 1.5, D) 2"


,field,value
0,topic,Логарифми
1,type,task
2,system_prompt,"Ти - вчитель математики. Створи математичний контент на задану тему відповідно до вказаного типу.\n\nТипи контенту:\n1. Задача: \n - description: умова задачі з одним питанням\n - solution: **нумерований** список кроків розв'язання\n - answer: число або рівняння\n\n2. Тест:\n - description: умова задачі з чотирма варіантами відповіді (A, B, C, D)\n - solution: **нумерований** список кроків для знаходження відповіді\n - answer: одна латинська літера (A, B, C або D)\n\nВідповідь надай у JSON форматі з трьома аттрибутами типу string: description, solution, answer. Не обгортай JSON у додаткові спецсимволи."
3,user_message,"Тема: Логарифми, Тип: Задача"
4,model_response,"{\n ""description"": ""Знайдіть значення x, якщо log₂(x) + log₂(8) = 5."",\n ""solution"": ""1. Використовуємо властивість логарифмів: log₂(x) + log₂(8) = log₂(x * 8).\n2. Переписуємо рівняння: log₂(x * 8) = 5.\n3. Застосовуємо визначення логарифма: x * 8 = 2^5.\n4. Обчислюємо 2^5: 2^5 = 32.\n5. Розв'язуємо рівняння x * 8 = 32: x = 32 / 8.\n6. Обчислюємо значення x: x = 4."",\n ""answer"": ""4""\n}"
5,sample_description,Обчисліть значення логарифма: $\log_2 32$.
6,sample_solution,"1. Визначте означення логарифма: це степінь, до якого треба піднести основу (2), щоб отримати аргумент (32). 2. Перевірте степені двійки: $2^1=2$, $2^2=4$, $2^3=8$, $2^4=16$, $2^5=32$. 3. Оскільки $2^5 = 32$, значення логарифма дорівнює 5."
7,sample_answer,5
8,is_valid_json,True
9,generated_description,"Знайдіть значення x, якщо log₂(x) + log₂(8) = 5."


,field,value
0,topic,Функції
1,type,quiz
2,system_prompt,"Ти - вчитель математики. Створи математичний контент на задану тему відповідно до вказаного типу.\n\nТипи контенту:\n1. Задача: \n - description: умова задачі з одним питанням\n - solution: **нумерований** список кроків розв'язання\n - answer: число або рівняння\n\n2. Тест:\n - description: умова задачі з чотирма варіантами відповіді (A, B, C, D)\n - solution: **нумерований** список кроків для знаходження відповіді\n - answer: одна латинська літера (A, B, C або D)\n\nВідповідь надай у JSON форматі з трьома аттрибутами типу string: description, solution, answer. Не обгортай JSON у додаткові спецсимволи."
3,user_message,"Тема: Функції, Тип: Тест"
4,model_response,"{\n ""description"": ""Знайдіть значення функції f(x) = 2x^2 - 3x + 5 при x = 3. \nA) 20 \nB) 23 \nC) 26 \nD) 29"",\n ""solution"": ""1. Підставте значення x = 3 у функцію f(x) = 2x^2 - 3x + 5.\n2. Обчисліть 2(3)^2: 2 * 9 = 18.\n3. Обчисліть -3 * 3: -9.\n4. Додайте результати: 18 - 9 + 5.\n5. Обчисліть 18 - 9 = 9.\n6. Додайте 5 до 9: 9 + 5 = 14.\n7. Перевірте варіанти відповіді: жоден з варіантів не відповідає обчисленому значенню, тому перевірте ще раз обчислення."",\n ""answer"": ""A""\n}"
5,sample_description,Яка з наведених функцій є лінійною? \n A) $y = x^2$ \n B) $y = \frac{1}{x}$ \n C) $y = 2x + 5$ \n D) $y = \sqrt{x}$
6,sample_solution,"1. Згадайте означення лінійної функції: $y = kx + b$, де $x$ у першому степені. 2. Проаналізуйте варіант A: $x^2$ — квадратична функція. 3. Проаналізуйте варіант B: $1/x$ — обернена пропорційність. 4. Проаналізуйте варіант C: відповідає вигляду $kx + b$ ($k=2, b=5$). 5. Оберіть варіант C."
7,sample_answer,C
8,is_valid_json,True
9,generated_description,Знайдіть значення функції f(x) = 2x^2 - 3x + 5 при x = 3. \nA) 20 \nB) 23 \nC) 26 \nD) 29
